# AIDA Nanopublication Generator

Creates AIDA (Atomic, Independent, Declarative, Absolute) sentence nanopublications from a JSON configuration file.

**Template:** [AIDA Sentence Template](https://w3id.org/np/RALmXhDw3rHcMveTgbv8VtWxijUHwnSqhCmtJFIPKWVaA)

## AIDA Sentences
- **A**tomic: A sentence describing one thought that cannot be further broken down
- **I**ndependent: A sentence that can stand on its own, without external references
- **D**eclarative: A complete sentence ending with a full stop that could be true or false
- **A**bsolute: A sentence describing the core of a claim ignoring uncertainty

In [19]:
import json
import sys
import urllib.parse
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd().parent))

from nanopub_utils import (
    NanopubGenerator, load_config, save_nanopub,
    make_literal, PREFIXES
)

In [20]:
# AIDA-specific constants
HYCL_AIDA_SENTENCE = "http://purl.org/petapico/o/hycl#AIDA-Sentence"
HYCL_NS = "http://purl.org/petapico/o/hycl"
CITO_OBTAINS_SUPPORT = "http://purl.org/spar/cito/obtainsSupportFrom"
SCHEMA_ABOUT = "http://schema.org/about"
SKOS_RELATED = "http://www.w3.org/2004/02/skos/core#related"
DCT_IS_PART_OF = "http://purl.org/dc/terms/isPartOf"
AIDA_NS = "http://purl.org/aida/"

class AidaNanopubGenerator(NanopubGenerator):
    """Generator for AIDA sentence nanopublications matching Nanodash format."""
    
    def __init__(self, config: dict, nanopub_config: dict):
        # Merge metadata with individual nanopub config
        merged_config = {
            **config.get('metadata', {}),
            **nanopub_config,
            'template_uri': config.get('template_uri')
        }
        super().__init__(merged_config)
        
        # Add nanopub types for AIDA
        self.add_nanopub_type(HYCL_AIDA_SENTENCE)
        self.add_nanopub_type(HYCL_NS)
        
        # isPartOf goes in assertion for AIDA (links the sentence entity)
        self.is_part_of_in_assertion = True
    
    def generate_assertion(self) -> str:
        """Generate the AIDA assertion graph using full URIs."""
        aida_sentence = self.config['aida_sentence']
        topics = self.config.get('topics', [])
        publication = self.config.get('related_publication')
        dataset = self.config.get('related_dataset')
        project = self.config.get('related_project')
        is_part_of = self.config.get('is_part_of')
        
        # Create URL-encoded AIDA sentence URI
        aida_uri = AIDA_NS + urllib.parse.quote(aida_sentence, safe='')
        
        # Set this as the introduced URI
        self.set_introduces(aida_uri)
        
        # Truncate label if needed (like Nanodash does)
        label_text = aida_sentence
        if len(label_text) > 100:
            label_text = label_text[:100] + "..."
        self.config['label'] = f"AIDA sentence: {label_text}"
        
        lines = [f"{self.sub_prefix}:assertion {{"]
        
        # Build the assertion using full URIs (not prefixed)
        # Collect all predicates for the AIDA sentence
        predicates = []
        predicates.append(f"    a <{HYCL_AIDA_SENTENCE}>")
        
        # Add related publication(s)
        if publication:
            pub_uri = publication if publication.startswith('http') else f"https://doi.org/{publication}"
            predicates.append(f"    <{CITO_OBTAINS_SUPPORT}> <{pub_uri}>")
        
        # Add related dataset
        if dataset:
            predicates.append(f"    <{CITO_OBTAINS_SUPPORT}> <{dataset}>")
        
        # Add topic relationships
        for topic in topics:
            topic_uri = topic.get('uri', '')
            topic_label = topic.get('label', '')
            if topic_uri:
                predicates.append(f"    <{SCHEMA_ABOUT}> <{topic_uri}>")
                # Track Wikidata labels for pubinfo
                if topic_label:
                    self.add_wikidata_label(topic_uri, topic_label)
        
        # Add related project
        if project:
            predicates.append(f"    <{SKOS_RELATED}> <{project}>")
        
        # Add isPartOf (link AIDA sentence to systematic review)
        if is_part_of and is_part_of.get('uri'):
            predicates.append(f"    <{DCT_IS_PART_OF}> <{is_part_of['uri']}>")
        
        # Format with proper semicolons
        lines.append(f"  <{aida_uri}>")
        for i, pred in enumerate(predicates):
            if i < len(predicates) - 1:
                lines.append(f"{pred};")
            else:
                lines.append(f"{pred} .")
        
        lines.append("}")
        return "\n".join(lines)

In [21]:
# Configuration
CONFIG_FILE = "../config/vbae208/vbae208_aida.json"  # Change this to use different config
#CONFIG_FILE = "../config/clenet2025/clenet2025_aida.json"  # Change this to use different config
OUTPUT_DIR = "../output/aida"

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [22]:
# Load configuration
config = load_config(CONFIG_FILE)

print(f"Source paper: {config['metadata']['source_paper']['title']}")
print(f"Number of AIDA nanopublications to generate: {len(config['nanopublications'])}")
if config['metadata'].get('is_part_of'):
    print(f"Part of: {config['metadata']['is_part_of']['label']}")
print()

for i, np_config in enumerate(config['nanopublications'], 1):
    print(f"{i}. {np_config['aida_sentence'][:80]}...")

Source paper: QOMIC: Quantum Optimization for Motif Identification in Networks
Number of AIDA nanopublications to generate: 4
Part of: Quantum Computing Applications for Biodiversity Research - Systematic Review

1. Quantum approximate optimization algorithms outperform classical methods for dis...
2. The QOMIC algorithm achieves up to 63.6% F1 score improvement over classical met...
3. Quantum annealing provides computational advantages for NP-hard network motif id...
4. Network motif analysis reveals conserved regulatory patterns in disease-associat...


In [23]:
# Generate nanopublications
generated_files = []

for np_config in config['nanopublications']:
    # Create generator
    generator = AidaNanopubGenerator(config, np_config)
    
    # Generate nanopub content
    nanopub_content = generator.generate()
    
    # Save to file
    output_file = f"{OUTPUT_DIR}/{np_config['id']}.trig"
    save_nanopub(nanopub_content, output_file)
    generated_files.append(output_file)
    
    print(f"Generated: {output_file}")

print(f"\nTotal generated: {len(generated_files)} nanopublications")

Generated: ../output/aida/aida_qomic_01.trig
Generated: ../output/aida/aida_qomic_02.trig
Generated: ../output/aida/aida_qomic_03.trig
Generated: ../output/aida/aida_qomic_04.trig

Total generated: 4 nanopublications


In [24]:
# Preview first generated nanopublication
if generated_files:
    print(f"Preview of {generated_files[0]}:\n")
    print("=" * 80)
    with open(generated_files[0], 'r') as f:
        print(f.read())

Preview of ../output/aida/aida_qomic_01.trig:

@prefix this: <https://w3id.org/np/RAe3eee6ed88665f1d2258d91ebd7088790a4d544c3f6> .
@prefix sub: <https://w3id.org/np/RAe3eee6ed88665f1d2258d91ebd7088790a4d544c3f6/> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix nt: <https://w3id.org/np/o/ntemplate/> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sub:Head {
  this: a np:Nanopublication ;
    np:hasAssertion sub:assertion ;
    np:hasProvenance sub:provenance ;
    np:hasPublicationInfo sub:pubinfo .
}

sub:assertion {
  <http://purl.org/aida/Quantum%20approximate%20optimization%20algorithms%20outperform%20classical%20methods%20for%20disjoint%20n

## JSON Config Structure

```json
{
  "metadata": {
    "source_paper": { "title": "...", "doi": "..." },
    "creator_orcid": "0000-0000-0000-0000",
    "creator_name": "Your Name",
    "is_part_of": {
      "uri": "https://w3id.org/np/YOUR-REVIEW-URI",
      "label": "Your Systematic Review Title"
    }
  },
  "nanopublications": [
    {
      "id": "aida_example_01",
      "aida_sentence": "Your AIDA sentence here.",
      "related_publication": "10.xxxx/xxxxx",
      "topics": [
        {"uri": "http://www.wikidata.org/entity/Qxxxxx", "label": "topic name"}
      ]
    }
  ]
}
```

The `is_part_of` creates a `dct:isPartOf` triple in the **assertion** linking the AIDA sentence entity to your systematic review.